In [1]:
%%capture
!pip install datasets transformers
!pip install sentencepiece

In [2]:
import datasets
import transformers
import pandas as pd
import numpy as np
import torch
from IPython.display import display, HTML
from transformers import pipeline

In [3]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# Final Performance (Scraped Recent News Dataset)

**Please note that the GPT2 results are generated in the Baseline_gpt2.ipynb file.**

## BART

In [8]:
# Load Trained Model
hub_model_id = "Alred/bart-base-finetuned-summarization-cnn-ver1.3"
summarizer = pipeline("summarization", model=hub_model_id, num_beams=6, no_repeat_ngram_size=3, temperature=2., do_sample=True)

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [9]:
def preprocess_content(content, max_length=1024):
    input = summarizer.tokenizer(content, max_length=max_length, truncation=True)['input_ids']
    return summarizer.tokenizer.decode(input, skip_special_tokens=True)

In [10]:
def run_sum_text(summarizer, choice='BART'):
    """
    Subset Version
    """

    # Load Data 
    df_new_data = pd.read_csv("../Data/recent_news/bbc_news_updated.csv", index_col=0)

    # Preprocess Input Text if needed
    if choice == 'BART':
        df_new_data['Cleaned_Content'] = df_new_data["Content"].map(lambda x: preprocess_content(x))
    
    # Perform Summarization
    review = df_new_data["Content"].tolist()
    if choice == "BART":
        review = df_new_data["Cleaned_Content"].tolist()
    
    summary = summarizer(review)

    # Postprocess generated summaries
    cleaned_summary = list(map(lambda x: x['summary_text'], summary))
    output_df = pd.DataFrame({"Text": df_new_data['Content'], "Original Summary": df_new_data['Original Summary'], "Generated_summary": cleaned_summary})

    return output_df

In [11]:
def display_outcome(df, n):
    display(HTML(df.head(n).to_html()))

In [12]:
output_df = run_sum_text(summarizer)

In [ ]:
output_df_bart = output_df.drop(['Original Summary'], axis=1)
output_df_bart.columns = ['Text', 'Generated_Summary_Bart']
output_df_bart

,Text,Generated_Summary_Bart
0,The takeover of Britain's largest microchip pl...,Nexperia must sell 86% of its stake to mitigat...
1,Elon Musk has told Twitter staff that they mus...,Elon Musk tells Twitter staff that they must w...
2,Chancellor Jeremy Hunt will pledge to face int...,"UK chancellor to announce widespread cuts, tax..."
3,The first sign of job cuts at Amazon came from...,A number of companies are announcing job cuts ...
4,UK-based food delivery app Deliveroo says it i...,UK-based food delivery app Deliveroo says it i...
...,...,...
69,"""The Wagatha Christie case has mystery, suspen...",Liv Hennessy plays the court battle in the new...
70,A court in India has granted bail to Bollywood...,Bombay court denies charges against Jacqueline...
71,Michelle Obama has admitted she struggles with...,"Michelle Obama says people are ""locked in our ..."
72,The BBC is celebrating the centenary of its fi...,New research reveals early BBC moments came fr...


## T5

In [ ]:
# Load Trained Model
hub_model_id = "Alred/t5-base-finetuned-summarization-cnn-ver2"
summarizer = pipeline("summarization", model=hub_model_id, temperature=2., do_sample=True)

In [ ]:
summarizer.model.config

T5Config {
  "_name_or_path": "Alred/t5-base-finetuned-summarization-cnn-ver2",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "early_stopping": true,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "length_penalty": 2.0,
  "max_length": 200,
  "min_length": 30,
  "model_type": "t5",
  "n_positions": 512,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "prefix": "summarize: ",
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
    

In [ ]:
output_df2 = run_sum_text(summarizer, choice="T5")

Token indices sequence length is longer than the specified maximum sequence length for this model (670 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
output_df2

,Text,Original Summary,Generated_summary
0,The takeover of Britain's largest microchip pl...,The UK government says Nexperia's takeover of ...,UK firm's decision to buy the UK's biggest mic...
1,Elon Musk has told Twitter staff that they mus...,Elon Musk says workers at the social media fir...,Elon Musk says Twitter will need to be extreme...
2,Chancellor Jeremy Hunt will pledge to face int...,Pensions and benefits will rise with prices bu...,Chancellor Hunt is expected to confirm widespr...
3,The first sign of job cuts at Amazon came from...,Thousands of redundancies have been announced ...,"More than 120,000 jobs lost across the tech in..."
4,UK-based food delivery app Deliveroo says it i...,The firm struggled to compete with rivals as w...,Deliveroo is being placed into voluntary admin...
...,...,...,...
69,"""The Wagatha Christie case has mystery, suspen...",NaN,"Playground Liv Hennessy waded through 1,200 pa..."
70,A court in India has granted bail to Bollywood...,NaN,Bollywood actress Jacqueline Fernandez alleged...
71,Michelle Obama has admitted she struggles with...,NaN,Michelle Obama speaks about her fear of being ...
72,The BBC is celebrating the centenary of its fi...,NaN,The BBC celebrated the centenary of it first o...


In [ ]:
output_df_t5 = output_df2.drop(['Original Summary'], axis=1)
output_df_t5.columns = ['Text', 'Generated_Summary_T5']
output_df_t5

,Text,Generated_Summary_T5
0,The takeover of Britain's largest microchip pl...,UK firm's decision to buy the UK's biggest mic...
1,Elon Musk has told Twitter staff that they mus...,Elon Musk says Twitter will need to be extreme...
2,Chancellor Jeremy Hunt will pledge to face int...,Chancellor Hunt is expected to confirm widespr...
3,The first sign of job cuts at Amazon came from...,"More than 120,000 jobs lost across the tech in..."
4,UK-based food delivery app Deliveroo says it i...,Deliveroo is being placed into voluntary admin...
...,...,...
69,"""The Wagatha Christie case has mystery, suspen...","Playground Liv Hennessy waded through 1,200 pa..."
70,A court in India has granted bail to Bollywood...,Bollywood actress Jacqueline Fernandez alleged...
71,Michelle Obama has admitted she struggles with...,Michelle Obama speaks about her fear of being ...
72,The BBC is celebrating the centenary of its fi...,The BBC celebrated the centenary of it first o...


In [ ]:
final_df = output_df_bart.merge(output_df_t5, on='Text')
final_df

,Text,Generated_Summary_Bart,Generated_Summary_T5
0,The takeover of Britain's largest microchip pl...,Nexperia must sell 86% of its stake to mitigat...,Nexperia must sell 86% of its stake to mitigat...
1,Elon Musk has told Twitter staff that they mus...,Elon Musk tells Twitter staff that they must w...,Elon Musk tells Twitter staff that they must w...
2,Chancellor Jeremy Hunt will pledge to face int...,"UK chancellor to announce widespread cuts, tax...","UK chancellor to announce widespread cuts, tax..."
3,The first sign of job cuts at Amazon came from...,A number of companies are announcing job cuts ...,A number of companies are announcing job cuts ...
4,UK-based food delivery app Deliveroo says it i...,UK-based food delivery app Deliveroo says it i...,UK-based food delivery app Deliveroo says it i...
...,...,...,...
69,"""The Wagatha Christie case has mystery, suspen...",Liv Hennessy plays the court battle in the new...,Liv Hennessy plays the court battle in the new...
70,A court in India has granted bail to Bollywood...,Bombay court denies charges against Jacqueline...,Bombay court denies charges against Jacqueline...
71,Michelle Obama has admitted she struggles with...,"Michelle Obama says people are ""locked in our ...","Michelle Obama says people are ""locked in our ..."
72,The BBC is celebrating the centenary of its fi...,New research reveals early BBC moments came fr...,New research reveals early BBC moments came fr...


In [ ]:
final_df.to_csv("bart_t5_v2.csv")

In [ ]:
display_outcome(final_df, 5)

## Baseline1-First 3 Sentences

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import os

import nltk

nltk.download("punkt")

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df_for_baseline = pd.read_csv("../Data/recent_news/bbc_news_updated.csv")
df_for_baseline = df_for_baseline[['Content']]
df_for_baseline 

,Content
0,The takeover of Britain's largest microchip pl...
1,Elon Musk has told Twitter staff that they mus...
2,Chancellor Jeremy Hunt will pledge to face int...
3,The first sign of job cuts at Amazon came from...
4,UK-based food delivery app Deliveroo says it i...
...,...
69,"""The Wagatha Christie case has mystery, suspen..."
70,A court in India has granted bail to Bollywood...
71,Michelle Obama has admitted she struggles with...
72,The BBC is celebrating the centenary of its fi...


In [ ]:
def first_three_sent_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [ ]:
def display_outcome(df, n):
    display(HTML(df.head(n).to_html()))

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [ ]:
import re
def clean_data(text):
  text = text.lower()
  text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
  
  text = re.sub("cnn", "", text)
  text=re.sub(r'\n',' ', text)
  text=re.sub(r'>',' ', text)
  text=re.sub(r'<',' ', text)
  text=re.sub(r'LRB',' ', text)
  text=re.sub(r'RRB',' ', text)
  text = re.sub(r'[" "]+', " ", text)
  text=re.sub(r'-- ',' ', text)
  text=re.sub(r"([?!¿])", r" \1 ", text)
  text=re.sub(r'-',' ', text)
  text=text.replace('/',' ')
  text=re.sub(r'\s+', ' ', text)
  text = re.sub('[^A-Za-z0-9.,]+', ' ', text)
  text = re.sub(r'([.?!]+) *', r' \1 ', text)
  return text

In [ ]:
df_for_baseline['Cleaned_Content'] = df_for_baseline['Content'].map(lambda x: clean_data(x))
df_for_baseline

,Content,Cleaned_Content
0,The takeover of Britain's largest microchip pl...,the takeover of britain s largest microchip pl...
1,Elon Musk has told Twitter staff that they mus...,elon musk has told twitter staff that they mus...
2,Chancellor Jeremy Hunt will pledge to face int...,chancellor jeremy hunt will pledge to face int...
3,The first sign of job cuts at Amazon came from...,the first sign of job cuts at amazon came from...
4,UK-based food delivery app Deliveroo says it i...,uk based food delivery app deliveroo says it i...
...,...,...
69,"""The Wagatha Christie case has mystery, suspen...","the wagatha christie case has mystery, suspen..."
70,A court in India has granted bail to Bollywood...,a court in india has granted bail to bollywood...
71,Michelle Obama has admitted she struggles with...,michelle obama has admitted she struggles with...
72,The BBC is celebrating the centenary of its fi...,the bbc is celebrating the centenary of its fi...


In [ ]:
df_for_baseline['generated_summary'] = df_for_baseline['Cleaned_Content'].map(lambda x: first_three_sent_summary(x))
df_for_baseline

,Content,Cleaned_Content,generated_summary
0,The takeover of Britain's largest microchip pl...,the takeover of britain s largest microchip pl...,the takeover of britain s largest microchip pl...
1,Elon Musk has told Twitter staff that they mus...,elon musk has told twitter staff that they mus...,elon musk has told twitter staff that they mus...
2,Chancellor Jeremy Hunt will pledge to face int...,chancellor jeremy hunt will pledge to face int...,chancellor jeremy hunt will pledge to face int...
3,The first sign of job cuts at Amazon came from...,the first sign of job cuts at amazon came from...,the first sign of job cuts at amazon came from...
4,UK-based food delivery app Deliveroo says it i...,uk based food delivery app deliveroo says it i...,uk based food delivery app deliveroo says it i...
...,...,...,...
69,"""The Wagatha Christie case has mystery, suspen...","the wagatha christie case has mystery, suspen...","the wagatha christie case has mystery, suspen..."
70,A court in India has granted bail to Bollywood...,a court in india has granted bail to bollywood...,a court in india has granted bail to bollywood...
71,Michelle Obama has admitted she struggles with...,michelle obama has admitted she struggles with...,michelle obama has admitted she struggles with...
72,The BBC is celebrating the centenary of its fi...,the bbc is celebrating the centenary of its fi...,the bbc is celebrating the centenary of its fi...


In [ ]:
display_outcome(df_for_baseline, 5)

In [ ]:
df_baseline = df_for_baseline[['Content', 'generated_summary']]
df_baseline.columns = ['Text', "Generated_Summary_First3"]
df_baseline

,Text,Generated_Summary_First3
0,The takeover of Britain's largest microchip pl...,the takeover of britain s largest microchip pl...
1,Elon Musk has told Twitter staff that they mus...,elon musk has told twitter staff that they mus...
2,Chancellor Jeremy Hunt will pledge to face int...,chancellor jeremy hunt will pledge to face int...
3,The first sign of job cuts at Amazon came from...,the first sign of job cuts at amazon came from...
4,UK-based food delivery app Deliveroo says it i...,uk based food delivery app deliveroo says it i...
...,...,...
69,"""The Wagatha Christie case has mystery, suspen...","the wagatha christie case has mystery, suspen..."
70,A court in India has granted bail to Bollywood...,a court in india has granted bail to bollywood...
71,Michelle Obama has admitted she struggles with...,michelle obama has admitted she struggles with...
72,The BBC is celebrating the centenary of its fi...,the bbc is celebrating the centenary of its fi...


In [ ]:
df_baseline.to_csv("baseline_first3.csv")

# Final Results

In [ ]:
df1 = pd.read_csv("bart_t5_v2.csv", index_col=0)
# df1.drop("Unnamed: 0.1", axis=1, inplace=True)
df1

,Text,Generated_Summary_Bart,Generated_Summary_T5
0,The takeover of Britain's largest microchip pl...,Nexperia must sell 86% of its stake to mitigat...,UK firm's decision to buy the UK's biggest mic...
1,Elon Musk has told Twitter staff that they mus...,Elon Musk tells Twitter staff that they must w...,Elon Musk says Twitter will need to be extreme...
2,Chancellor Jeremy Hunt will pledge to face int...,"UK chancellor to announce widespread cuts, tax...",Chancellor Hunt is expected to confirm widespr...
3,The first sign of job cuts at Amazon came from...,A number of companies are announcing job cuts ...,"More than 120,000 jobs lost across the tech in..."
4,UK-based food delivery app Deliveroo says it i...,UK-based food delivery app Deliveroo says it i...,Deliveroo is being placed into voluntary admin...
...,...,...,...
69,"""The Wagatha Christie case has mystery, suspen...",Liv Hennessy plays the court battle in the new...,"Playground Liv Hennessy waded through 1,200 pa..."
70,A court in India has granted bail to Bollywood...,Bombay court denies charges against Jacqueline...,Bollywood actress Jacqueline Fernandez alleged...
71,Michelle Obama has admitted she struggles with...,"Michelle Obama says people are ""locked in our ...",Michelle Obama speaks about her fear of being ...
72,The BBC is celebrating the centenary of its fi...,New research reveals early BBC moments came fr...,The BBC celebrated the centenary of it first o...


In [ ]:
df2 = pd.read_csv("baseline_first3.csv", index_col=0)
df2

,Text,Generated_Summary_First3
0,The takeover of Britain's largest microchip pl...,the takeover of britain s largest microchip pl...
1,Elon Musk has told Twitter staff that they mus...,elon musk has told twitter staff that they mus...
2,Chancellor Jeremy Hunt will pledge to face int...,chancellor jeremy hunt will pledge to face int...
3,The first sign of job cuts at Amazon came from...,the first sign of job cuts at amazon came from...
4,UK-based food delivery app Deliveroo says it i...,uk based food delivery app deliveroo says it i...
...,...,...
69,"""The Wagatha Christie case has mystery, suspen...","the wagatha christie case has mystery, suspen..."
70,A court in India has granted bail to Bollywood...,a court in india has granted bail to bollywood...
71,Michelle Obama has admitted she struggles with...,michelle obama has admitted she struggles with...
72,The BBC is celebrating the centenary of its fi...,the bbc is celebrating the centenary of its fi...


In [ ]:
df3 = df1.merge(df2, on='Text')
df3

,Text,Generated_Summary_Bart,Generated_Summary_T5,Generated_Summary_First3
0,The takeover of Britain's largest microchip pl...,Nexperia must sell 86% of its stake to mitigat...,UK firm's decision to buy the UK's biggest mic...,the takeover of britain s largest microchip pl...
1,Elon Musk has told Twitter staff that they mus...,Elon Musk tells Twitter staff that they must w...,Elon Musk says Twitter will need to be extreme...,elon musk has told twitter staff that they mus...
2,Chancellor Jeremy Hunt will pledge to face int...,"UK chancellor to announce widespread cuts, tax...",Chancellor Hunt is expected to confirm widespr...,chancellor jeremy hunt will pledge to face int...
3,The first sign of job cuts at Amazon came from...,A number of companies are announcing job cuts ...,"More than 120,000 jobs lost across the tech in...",the first sign of job cuts at amazon came from...
4,UK-based food delivery app Deliveroo says it i...,UK-based food delivery app Deliveroo says it i...,Deliveroo is being placed into voluntary admin...,uk based food delivery app deliveroo says it i...
...,...,...,...,...
69,"""The Wagatha Christie case has mystery, suspen...",Liv Hennessy plays the court battle in the new...,"Playground Liv Hennessy waded through 1,200 pa...","the wagatha christie case has mystery, suspen..."
70,A court in India has granted bail to Bollywood...,Bombay court denies charges against Jacqueline...,Bollywood actress Jacqueline Fernandez alleged...,a court in india has granted bail to bollywood...
71,Michelle Obama has admitted she struggles with...,"Michelle Obama says people are ""locked in our ...",Michelle Obama speaks about her fear of being ...,michelle obama has admitted she struggles with...
72,The BBC is celebrating the centenary of its fi...,New research reveals early BBC moments came fr...,The BBC celebrated the centenary of it first o...,the bbc is celebrating the centenary of its fi...


In [ ]:
df3.to_csv("results_summary.csv")